In [26]:
# !pip install selenium

In [27]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from collections import defaultdict
from time import sleep
import pandas as pd
import numpy as np

In [28]:
def return_element_values(object, chrome): #returns a list of the values of features
    element_values = {}
    all_element_values = []
    for element in chrome.find_elements(By.TAG_NAME, object):
        element_values['tag'] = element.tag_name
        element_values['size'] = element.size
        element_values['location'] = element.location
        element_values['text'] = element.text
        try:
            element_values['first_class_element'] = chrome.find_element(By.CLASS_NAME, element.get_dom_attribute("class"))
        except (TimeoutException, StaleElementReferenceException, NoSuchElementException):
            element_values['first_class_element'] = np.nan
        for col in ['id','name','class']:
            try:
                # WebDriverWait(chrome).until(EC.presence_of_element_located((By.ID, element.id)))
                attribute = element.get_attribute(col)
                if attribute.strip():
                    element_values[col] = attribute
                
            except (TimeoutException, StaleElementReferenceException, NoSuchElementException):
                element_values[col] = np.nan
                
        all_element_values.append(element_values)
    return all_element_values

In [29]:
def make_dataset(url, columns = ['tag', 'name', 'class','id', 'text']): #Creates dataset using return_element_values(object, chrome)
    with webdriver.Chrome() as chrome:
        chrome.maximize_window()
        chrome.get(url) #opens a url in chrome
        WebDriverWait(chrome,5).until(EC.presence_of_all_elements_located((By.TAG_NAME,'*')))
        object_values = []
        for object in ['button','form','field','input', 'nav']:
            # try:
            object_values.append(pd.DataFrame(return_element_values(object, chrome), columns= columns).fillna(np.nan))
            # except (StaleElementReferenceException, NoSuchElementException) as e:
            #     continue
        return pd.concat(object_values)
        # return pd.concat(map(lambda object: pd.DataFrame(return_element_values(object, chrome), columns= columns), ['button','form','field']))


In [30]:
###########driver code
web_data = pd.DataFrame(columns = ['tag', 'name', 'class','id', 'text','location','size', 'first_class_element'])
for url in [r"https://www.reddit.com/r/learnprogramming/top/?t=month",r'https://www.mercadolivre.com.br/#from=homecom',r'https://www.indeed.com/',r'https://www.tripadvisor.com/',r'https://www.yellowpages.com/',r'https://www.ebay.com/',r'https://www.ebay.com/b/PC-Gaming/bn_7000259657']:
    web_data = pd.concat( [web_data, make_dataset(url, web_data.columns)])
print(web_data.info())
web_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 6
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   tag                  241 non-null    object
 1   name                 31 non-null     object
 2   class                235 non-null    object
 3   id                   156 non-null    object
 4   text                 241 non-null    object
 5   location             241 non-null    object
 6   size                 241 non-null    object
 7   first_class_element  8 non-null      object
dtypes: object(8)
memory usage: 16.9+ KB
None


,tag,name,class,id,text,location,size,first_class_element
0,button,NaN,_2iuoyPiKHN3kfOoeIQalDT _10BQ7pjWbeYP63SAPNS8T...,t3_yv51u5-overflow-menu,Back to Top,"{'x': 1085, 'y': 2482}","{'height': 32, 'width': 112}",NaN
1,button,NaN,_2iuoyPiKHN3kfOoeIQalDT _10BQ7pjWbeYP63SAPNS8T...,t3_yv51u5-overflow-menu,Back to Top,"{'x': 1085, 'y': 2482}","{'height': 32, 'width': 112}",NaN
2,button,NaN,_2iuoyPiKHN3kfOoeIQalDT _10BQ7pjWbeYP63SAPNS8T...,t3_yv51u5-overflow-menu,Back to Top,"{'x': 1085, 'y': 2482}","{'height': 32, 'width': 112}",NaN
3,button,NaN,_2iuoyPiKHN3kfOoeIQalDT _10BQ7pjWbeYP63SAPNS8T...,t3_yv51u5-overflow-menu,Back to Top,"{'x': 1085, 'y': 2482}","{'height': 32, 'width': 112}",NaN
4,button,NaN,_2iuoyPiKHN3kfOoeIQalDT _10BQ7pjWbeYP63SAPNS8T...,t3_yv51u5-overflow-menu,Back to Top,"{'x': 1085, 'y': 2482}","{'height': 32, 'width': 112}",NaN
...,...,...,...,...,...,...,...,...
2,input,_nkw,btn btn-prim gh-spr,crid,,"{'x': 0, 'y': 0}","{'height': 0, 'width': 0}",NaN
3,input,_nkw,btn btn-prim gh-spr,crid,,"{'x': 0, 'y': 0}","{'height': 0, 'width': 0}",NaN
4,input,_nkw,btn btn-prim gh-spr,crid,,"{'x': 0, 'y': 0}","{'height': 0, 'width': 0}",NaN
5,input,_nkw,btn btn-prim gh-spr,crid,,"{'x': 0, 'y': 0}","{'height': 0, 'width': 0}",NaN


In [31]:
web_data['first_class_element'][web_data.first_class_element.notna()]

0    <selenium.webdriver.remote.webelement.WebEleme...
0    <selenium.webdriver.remote.webelement.WebEleme...
0    <selenium.webdriver.remote.webelement.WebEleme...
0    <selenium.webdriver.remote.webelement.WebEleme...
1    <selenium.webdriver.remote.webelement.WebEleme...
2    <selenium.webdriver.remote.webelement.WebEleme...
3    <selenium.webdriver.remote.webelement.WebEleme...
0    <selenium.webdriver.remote.webelement.WebEleme...
Name: first_class_element, dtype: object